In [ ]:
import pandas as pd

pd.read_csv("tip25W_testdaten_template.csv")
df_prio = pd.read_csv("order_products__prior.csv")
df_train = pd.read_csv("order_products__train.csv")
df_products = pd.read_csv("products.csv")

df_all = pd.concat([df_prio, df_train])

df_departments = pd.read_csv("departments.csv")
df_departments = df_departments[df_departments["department_id"].between(1,5)]
df_departments

Aufgabe 1:
*Berechnen Sie die 5x5 Matrix M, die an Position m i,k jeweils angibt wie wahrscheinlich es ist, dass die
nachfolgende Bestellung eines Users mindestens einen Artikel aus Department k enthält
(, unter der Bedingung, dass die vorherige Bestellung desselben Users mindestens einen Artikel aus
dem Department mit ID i enthalten hat. (
Um die Matrix überschaubar zu halten, beschränkt sich die Aufgabe auf die
Departments 1 bis 5. Die wahren Wahrscheinlichkeiten sind natürlich
unbekannt; sie müssen also nur einen sinnvollen Schätzwert berechnen. Bei
Matrizen wird der erste Index nach unten, der zweite nach rechts
dargestellt.*

Also:
Wenn ein User etwas aus Department i gekauft hat: Wie wahrscheinlich ist es dann, dass er in seiner nächsten Bestellung etwas aus Department k kauft?

m i,k = P(nächste Bestellung enthält Department k | vorherige Bestellung enthielt Department i)

diese Bestellung aus k ∩ letze Bestellung aus k
_______________________________________________
letzte Bestellung aus k


| i, k    | 1 | 2 | 3 | 4 | 5 |
| -------- | ------- | ------- | ------- | ------- | ------- |
| 1        |   |   |   |   |   |
| 2        |   |   |   |   |   |
| 3        |   |   |   |   |   |
| 4        |   |   |   |   |   |
| 5        |   |   |   |   |   |

In [ ]:
import pandas as pd
overview = pd.merge(df_all[:100][['order_id', 'product_id']], df_products[['product_id', 'department_id']], on="product_id", how="left")
overview.head(5)

In [ ]:
from IPython.display import display

print("DF order Prio: ")
display(df_prio.head(1))

print("DF order train: ")
display(df_train.head(1))

print("DF orders:")
df_orders = pd.read_csv("orders.csv")
display(df_orders.head(11))

print("DF products:")
display(df_products.head(1))

print("Departments:")
df_dep = pd.read_csv("departments.csv")
display(df_dep.head(1))

In [ ]:
# Sort so latest orders are first per user
df_sorted = df_orders.sort_values(['user_id', 'order_number'], ascending=[True, False])

latest_two_per_user = df_sorted.groupby('user_id').head(2).reset_index(drop=True)
latest_two_per_user.head(10)

In [ ]:
latest_two_per_user_small = latest_two_per_user[["user_id", "order_id", "eval_set", "order_number"]]
latest_two_per_user_small

df_all_small = df_all[["product_id", "order_id"]]
df_all_small

In [ ]:
orders_merged = pd.merge(latest_two_per_user_small, df_all_small[:100], on="order_id",how="left")
orders_merged.head(10)